In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from bitquant.data.data_client import DataClient
from bitquant.data.exchange import BinanceExchange
from bitquant.quantlib.evaluation.evaluator import Evaluator
from bitquant.quantlib.functions.functions import *
from bitquant.quantlib.strategy_engine import StrategyEngine
from bitquant.quantlib.signal_generation.factor_calculator import FactorCalculator, function_map
from bitquant.quantlib.signal_generation.factor_selector import FactorSelector
from bitquant.quantlib.signal_generation.factor_scaler import FactorScaler
from bitquant.quantlib.signal_generation.factor_aggregator import FactorAggregatorIC
from bitquant.base.pair import TRADABLE_PAIRS
from bitquant.base.protocol import SymbolValueDict, PortfolioRecord
from datetime import datetime
import pandas as pd

In [6]:
# symbols = list(TRADABLE_PAIRS)
symbols = ["ETHUSDT", "BTCUSDT", "BNBUSDT", "SOLUSDT"]
interval = "1h"
st = "2024-01-01 00:00:00"
et = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
data_client = DataClient(BinanceExchange)
symbol_info, data = data_client.run(symbols, interval, st, et)

factor_lis = ["ts_midpoint(ts_natr(high,low,close,7),14)", "ts_delta(dynamic_ts_max(ts_bbands(close,20),28),7)",
              "ts_midpoint(ts_ht_trendmode(close),21)"]

factor_calculator = FactorCalculator(function_map, different_axis=['ts', 'symbol', 'return_1'])
factor_scaler = FactorScaler(scaling_window=180, orthogonalize=False, orthogonal_method='symmetry',
                             ts_normalize=True, cross_section_normalize=False)
factor_selector = FactorSelector()
factor_aggregator = FactorAggregatorIC(training_window=90, rolling_type="avg", ic_type='pearson')

strategy_engine = StrategyEngine(init_factor_lis=factor_lis, factor_calculator=factor_calculator,
                                 factor_scaler=factor_scaler, factor_selector=factor_selector,
                                 factor_aggregator=factor_aggregator)
portfolio_weight = strategy_engine.run(data)
svdict = SymbolValueDict(portfolio_weight)
portfolio_record = PortfolioRecord(svdict=svdict)


/Users/andy/Desktop/crypto_park/bittensor/bitquant/bitquant/quantlib/signal_generation/factor_scaler.py:46: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  cls.normalized_data = cls.normalized_data.unstack().iloc[rolling_window - 1:].stack()
/Users/andy/Desktop/crypto_park/bittensor/bitquant/bitquant/quantlib/signal_generation/factor_scaler.py:48: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  how='all').fillna(0).stack()
/Users/andy/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, 

In [12]:
import time
# fake simulation

portfolio_record_lis = []
for _ in range(5):
    symbols = ["ETHUSDT", "BTCUSDT", "BNBUSDT", "SOLUSDT"]
    interval = "30m"
    st = "2024-03-01 00:00:00"
    et = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    data_client = DataClient(BinanceExchange)
    symbol_info, data = data_client.run(symbols, interval, st, et)
    
    factor_lis = ["ts_midpoint(ts_natr(high,low,close,7),14)", "ts_delta(dynamic_ts_max(ts_bbands(close,20),28),7)",
                  "ts_midpoint(ts_ht_trendmode(close),21)"]
    
    factor_calculator = FactorCalculator(function_map, different_axis=['ts', 'symbol', 'return_1'])
    factor_scaler = FactorScaler(scaling_window=180, orthogonalize=False, orthogonal_method='symmetry',
                                 ts_normalize=True, cross_section_normalize=False)
    factor_selector = FactorSelector()
    factor_aggregator = FactorAggregatorIC(training_window=90, rolling_type="avg", ic_type='pearson')
    
    strategy_engine = StrategyEngine(init_factor_lis=factor_lis, factor_calculator=factor_calculator,
                                     factor_scaler=factor_scaler, factor_selector=factor_selector,
                                     factor_aggregator=factor_aggregator)
    portfolio_weight = strategy_engine.run(data)
    svdict = SymbolValueDict(portfolio_weight)
    portfolio_record = PortfolioRecord(svdict=svdict)
    portfolio_record_lis.append(portfolio_record)
    time.sleep(3*60)

/Users/andy/Desktop/crypto_park/bittensor/bitquant/bitquant/quantlib/signal_generation/factor_scaler.py:46: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  cls.normalized_data = cls.normalized_data.unstack().iloc[rolling_window - 1:].stack()
/Users/andy/Desktop/crypto_park/bittensor/bitquant/bitquant/quantlib/signal_generation/factor_scaler.py:48: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  how='all').fillna(0).stack()
/Users/andy/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, 

### Validator

In [13]:
from bitquant.quantlib.evaluation.evaluator import Evaluator
from bitquant.data.utils import TimeUtils

In [14]:
flg = len(portfolio_record_lis) - 1
portfolio_weight_container =[]
ts_ms_container = []
for portfolio_record in portfolio_record_lis:
    # construct portfolio weight history
    portfolio_weight_container.append(portfolio_record.portfolio)
    # fake timestamp for test
    ts_ms_container.append(TimeUtils.now_in_ms() - flg * TimeUtils.interval_str_to_ms("30m"))
    flg -= 1

# retrive the 1-minute kline data for validation
ts_lis = [pd.to_datetime(ts_ms, unit="ms", utc=True).ceil("1min") for ts_ms in ts_ms_container]
portfolio_weight_df = pd.DataFrame(portfolio_weight_container, index=ts_lis)
symbols = portfolio_weight_df.columns.tolist()
interval = "1m"
st = TimeUtils.timestamp_to_dt_str(ts_lis[0], "%Y-%m-%d %H:%M:%S")
et = TimeUtils.timestamp_to_dt_str(ts_lis[-1] + TimeUtils.str_to_timedelta(interval), "%Y-%m-%d %H:%M:%S")
symbol_info, data = data_client.run(symbols=symbols, interval=interval, st=st, et=et)

In [21]:
portfolio_weight_matrix = portfolio_weight_df.to_numpy()
price_matrix = data.loc[[ts+TimeUtils.str_to_timedelta(interval) for ts in ts_lis], ["open", "low", "high", "close", "vwap"]].values.reshape(len(ts_lis), len(symbols), 5)

volume_precision = symbol_info.set_index("symbol").loc[symbols, "volume_precision"].values
min_notional = symbol_info.set_index("symbol").loc[symbols, "notional"].values

In [36]:
evaluator = Evaluator()
profit_ratio = evaluator.evaluate(portfolio_weight_matrix, price_matrix, init_asset_value=10000, taker_fee=0.0004,
                   volume_precision=volume_precision, min_notional=min_notional)
print(profit_ratio)

/Users/andy/Desktop/crypto_park/bittensor/bitquant/bitquant/quantlib/evaluation/evaluator.py:30: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  tradeAmountInUsdt = -1 * orders_v * present_price * np.where(trade_portfolio_weight > 0, 1 + taker_fee, 1 - taker_fee)


-0.008393290859999958